In [1]:
from functions import (prep, folding, inbu)
import copy
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import LSTM, Input
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# import session_info
# session_info.show()

In [2]:
np.random.seed(42)
filename = "running-example"
flattent_by = 'Orders'
ocel, act_dict, cust_dict = prep.prepare_flat_ocel(filename, flatten_on= flattent_by)
print(act_dict)
print(cust_dict)
drops_col_order = ["weight", "price", "Event_ID", 'Products']
enriched_log, single_log = prep.gen_enriched_single_plus_csv(OCEL = ocel,flatted_by = 'Orders',csvname = 'orders_complete', drops_col= drops_col_order)

{'place order': 'A', 'pick item': 'B', 'confirm order': 'C', 'item out of stock': 'D', 'reorder item': 'E', 'pay order': 'F', 'create package': 'G', 'send package': 'H', 'failed delivery': 'I', 'package delivered': 'J', 'payment reminder': 'K'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}


In [3]:
enriched_log =prep.gen_features(enriched_log)
divisor = np.mean(enriched_log['Time_Diff'])  # average time between events
divisor2 = np.mean(enriched_log['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(enriched_log['Remaining_Time'])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
enr_train, enr_test = folding.folding_train_test(enriched_log)

divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932


defining the features

In [4]:
act_feat = list(filter(lambda k: k.startswith('Act_') and not k.startswith('Next_Act_'), enr_train.columns))
target_act_feat = list(filter(lambda k: k.startswith('Next_Act_') and not k.startswith('Act_'), enr_train.columns))
act_feat.remove('Act_!')
cust_feat = list(filter(lambda k: 'Cust_' in k, enr_train.columns))
time_feat = ['Time_Diff', 'Time_Since_Start', 'Time_Since_Midnight','Weekday']
other_features = ['Amount_Items','In_Package','Position']
feature_select = act_feat + cust_feat + time_feat + other_features
print(f"Length of act_feat: {len(act_feat)}, Length of cust_feat: {len(cust_feat)}")

## define dimensions of inputs
traces, max_trace_length = prep.gen_traces_and_maxlength_of_trace(enriched_log)
target_act_length = len(target_act_feat)
number_of_train_cases = len(enr_train)
num_of_features = len(feature_select)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}")


Length of act_feat: 11, Length of cust_feat: 18
Number of train cases: 21620, Max trace length: 42, Number of features: 36


get the inputs 

In [5]:

X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(OCEL=enr_train,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                    divisor_next=divisor,
                                                                    divisor_since=divisor2,
                                                                    divisor_remaining=divisorTR)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}")

Shape of X_train: (21620, 42, 36)
This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): (21620, 42, 36) => True
Shape of y_train_a: (21620, 12), this matches the desired shape (number_of_train_cases, target_act_length): (21620, 12) => True
Shape of y_train_t: (21620,), this matches the desired shape (number_of_train_cases, ): (21620,) => True
Shape of y_train_tr: (21620,), this matches the desired shape (number_of_train_cases, ): (21620,) => True


In [59]:
pd.DataFrame(X[42,:,:])

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [35]:
np.random.seed(42)
filename = "running-example"
flattent_by = 'Orders'
ocel, act_dict, cust_dict = prep.prepare_flat_ocel(filename, flatten_on= flattent_by)
print(act_dict)
print(cust_dict)
drops_col_order = ["weight", "price", "Event_ID", 'Products']
enriched_log, single_log = prep.gen_enriched_single_plus_csv(OCEL = ocel,flatted_by = 'Orders',csvname = 'orders_complete', drops_col= drops_col_order)

lastcase = ''
line = ''
firstLine = True
lines = []
Ptimeseqs = []
Ptimeseqs2 = []
Ptimeseqs3 = []
Ptimeseqs4 = []
nb_itemseqs = []

Ptimes = []
Ptimes2 = []
Ptimes3 = []
Ptimes4 = []
nb_items = []
ascii_offset = 64
Pcasestarttime = None
Plasteventtime = None

for index, row in enriched_log.iterrows():
    t = row['Timestamp']
    nb = row['Amount_Items']
    if row['Case_ID'] != lastcase:
        Stime = t
        Pcasestarttime = t
        Plasteventtime = t
        lastcase = row['Case_ID']
        if not firstLine:
            lines.append(line)
            Ptimeseqs.append(Ptimes)
            Ptimeseqs2.append(Ptimes2)
            Ptimeseqs3.append(Ptimes3)
            Ptimeseqs4.append(Ptimes4)
            nb_itemseqs.append(nb_items)

        line = ''
        Ptimes = []
        Ptimes2 = []
        Ptimes3 = []
        Ptimes4 = []
        nb_items = []

    line += row['Activity'] #unichr(int(row['Activity']) + ascii_offset)
    Ptimesincelastevent = t - Plasteventtime
    Ptimesincecasestart = t - Pcasestarttime
    midnight = t.replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = t - midnight
    Ptimediff = 86400 * Ptimesincelastevent.days + Ptimesincelastevent.seconds
    Ptimediff2 = 86400 * Ptimesincecasestart.days + Ptimesincecasestart.seconds
    Ptimediff3 = timesincemidnight.seconds
    Ptimediff4 = t.weekday()
    Ptimes.append(Ptimediff)
    Ptimes2.append(Ptimediff2)
    Ptimes3.append(Ptimediff3)
    Ptimes4.append(Ptimediff4)
    nb_items.append(nb)
    Plasteventtime = t
    firstLine = False

# Add last case
lines.append(line)
Ptimeseqs.append(Ptimes)
Ptimeseqs2.append(Ptimes2)
Ptimeseqs3.append(Ptimes3)
Ptimeseqs4.append(Ptimes4)
nb_itemseqs.append(nb_items)

PtimeseqsF = []
for seq in Ptimeseqs2:
    PtimeseqsF.append([seq[-1] - t for t in seq])



divisor = np.mean([item for sublist in Ptimeseqs for item in sublist])  # average time between events
divisor2 = np.mean([item for sublist in Ptimeseqs2 for item in sublist])  # average time between current and first events
divisorTR = np.mean([item for sublist in PtimeseqsF for item in sublist])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f'numoof lines {len(lines)}')
print(len(nb_itemseqs))
### folding the lists 
lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5, lines_t6 = folding.folding_based_arrays(lines,Ptimeseqs,Ptimeseqs2,Ptimeseqs3,Ptimeseqs4,nb_itemseqs,PtimeseqsF,seeded = 42)
lines = list(map(lambda x: x + '!', lines))  # put delimiter symbol
maxlen = max(map(lambda x: len(x), lines))  # find maximum line size

# next lines here to get all possible characters for events and annotate them with numbers
# chars = map(lambda x: set(x),lines)
chars = list(set().union(*map(lambda x: set(x), lines)))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

{'place order': 'A', 'pick item': 'B', 'confirm order': 'C', 'item out of stock': 'D', 'reorder item': 'E', 'pay order': 'F', 'create package': 'G', 'send package': 'H', 'failed delivery': 'I', 'package delivered': 'J', 'payment reminder': 'K'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}
divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932
numoof lines 2000
2000
total chars: 11, target chars: 12
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K'}


In [36]:
step = 1
sentences = []
softness = 0
next_chars = []
sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
sentences_t5 = []
sentences_t6 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []
next_chars_t5 = []
next_chars_t6 = []


for line, line_t, line_t2, line_t3, line_t4, line_t5, line_t6 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5, lines_t6):
    for i in range(0, len(line), step):
        if i == 0:
            continue

        # we add iteratively, first symbol of the line, then two first, three...

        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        sentences_t5.append(line_t5[0:i])
        sentences_t6.append(line_t6[0:i])
        next_chars.append(line[i])
        if i == len(line) - 1:  # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
            next_chars_t5.append(0)
            next_chars_t6.append(0)


        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
            next_chars_t5.append(line_t5[i])
            next_chars_t6.append(line_t6[i])

print('nb sequences:', len(sentences))

print('Vectorization...')
num_features = len(chars) + 6
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)
y_tr = np.zeros((len(sentences)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    leftpad = maxlen - len(sentence)
    next_t = next_chars_t[i]
    next_tr = next_chars_t6[i] # wrong in original code was t5

    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    sentence_t5 = sentences_t5[i]

    for t, char in enumerate(sentence):
        for c in chars:
            if c == char:  # this will encode present events to the right places
                X[i, t + leftpad, char_indices[c]] = 1
        X[i, t + leftpad, len(chars)] = t + 1
        X[i, t + leftpad, len(chars) + 1] = sentence_t[t] / divisor
        X[i, t + leftpad, len(chars) + 2] = sentence_t2[t] / divisor2
        X[i, t + leftpad, len(chars) + 3] = sentence_t3[t] / 86400
        X[i, t + leftpad, len(chars) + 4] = sentence_t4[t] / 7
        X[i, t + leftpad, len(chars) + 5] = sentence_t5[t]              # number of objects

    for c in target_chars:
        if c == next_chars[i]:
            y_a[i, target_char_indices[c]] = 1 - softness
        else:
            y_a[i, target_char_indices[c]] = softness / (len(target_chars) - 1)
    y_t[i] = next_t / divisor
    y_tr[i] = next_tr / divisorTR

    np.set_printoptions(threshold=sys.maxsize)


nb sequences: 21620
Vectorization...
num features: 17


In [ ]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y_a: {y_a.shape}")
print(f"Shape of y_t: {y_t.shape}")
print(f"Shape of y_tr: {y_tr.shape}")
# Reshape the 3D array into 2D
reshaped_data = X.reshape((-1, X.shape[-1]))

# Convert reshaped ndarray to DataFrame
df = pd.DataFrame(reshaped_data)

# Print the resulting DataFrame
display(df[77:85])#.drop_duplicates())
display(df[40:43])#.drop_duplicates())
# display(df[-43:])#.drop_duplicates())


In [ ]:
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(
    main_input)  # the shared layer
b1 = tf.keras.layers.BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in activity prediction
b2_1 = tf.keras.layers.BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time prediction
b2_2 = tf.keras.layers.BatchNormalization()(l2_2)
l2_3 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time remaining prediction
b2_3 = tf.keras.layers.BatchNormalization()(l2_3)
act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(
    b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

timeR_output = Dense(1, kernel_initializer='glorot_uniform', name='timeR_output')(b2_3)

model = Model(inputs=[main_input], outputs=[act_output, time_output, timeR_output])

model.compile(loss={'act_output': 'categorical_crossentropy', 'time_output': 'mae', 'timeR_output': 'mae'}, optimizer='nadam')

early_stopping = EarlyStopping(monitor='val_loss', patience=50)
model_checkpoint = ModelCheckpoint('./output_files/models/model_'+filename+'_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss',
                                   verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, verbose=0, mode='auto', min_delta=0.0001,
                               cooldown=0, min_lr=0)

history = model.fit(X, {'act_output': y_a, 'time_output': y_t, 'timeR_output': y_tr}, validation_split=0.2, verbose=2,
          callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)
# list all data in history

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()